In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/titanic/test.csv
kaggle/input/titanic/train.csv


In [120]:
train_data = pd.read_csv("kaggle/input/titanic/train.csv")
train_data.head()

test_data = pd.read_csv("kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [121]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder  


device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')


class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        out = self.model(x)
        return out

model = Model(5, 1024)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [122]:
def prep_data(data):
    X_Train = data.drop(["Name"], axis=1)
    X_Train = X_Train.drop(["Ticket"], axis=1)
    X_Train = X_Train.drop(["Cabin"], axis=1)
    X_Train = X_Train.drop(["Embarked"], axis=1)
    X_Train = X_Train.drop(["PassengerId"], axis=1)
    X_Train = X_Train.drop(["Fare"], axis=1)
    X_Train["Age"] = X_Train["Age"].fillna(X_Train["Age"].mean())
    try:
        X_Train = X_Train.drop(["Survived"], axis=1)
    except:
        pass

    le = LabelEncoder()
    X_Train["Sex"] = le.fit_transform(X_Train["Sex"])
    #X_Train["Age"] = le.fit_transform(X_Train["Age"])
    return X_Train

X_Train = prep_data(train_data)
X_Test = prep_data(test_data)
y = train_data["Survived"]

X_Train.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,1,22.0,1,0
1,1,0,38.0,1,0
2,3,0,26.0,0,0
3,1,0,35.0,1,0
4,3,1,35.0,0,0


In [123]:

epoch = 20
X_Train = torch.from_numpy(X_Train.to_numpy()).float()
X_Test = torch.from_numpy(X_Test.to_numpy()).float()
y = torch.from_numpy(y.to_numpy()).float()

train_data = DataLoader(TensorDataset(X_Train, y), batch_size=32, shuffle=True)
#test_data = DataLoader(TensorDataset(X_Test, y), batch_size=32, shuffle=True)

print(X_Train)

for n in range(epoch):
    for x, y_test in train_data:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = nn.functional.mse_loss(y_pred, y_test)
        loss.backward()
        optimizer.step()

    print("Epoch: ", n, "Loss: ", loss.item())

tensor([[ 3.0000,  1.0000, 22.0000,  1.0000,  0.0000],
        [ 1.0000,  0.0000, 38.0000,  1.0000,  0.0000],
        [ 3.0000,  0.0000, 26.0000,  0.0000,  0.0000],
        ...,
        [ 3.0000,  0.0000, 29.6991,  1.0000,  2.0000],
        [ 1.0000,  1.0000, 26.0000,  0.0000,  0.0000],
        [ 3.0000,  1.0000, 32.0000,  0.0000,  0.0000]])
Epoch:  0 Loss:  0.2328646332025528
Epoch:  1 Loss:  0.379334419965744
Epoch:  2 Loss:  0.4127996563911438
Epoch:  3 Loss:  0.22731132805347443
Epoch:  4 Loss:  0.4780407249927521
Epoch:  5 Loss:  0.22036288678646088
Epoch:  6 Loss:  0.3055078089237213
Epoch:  7 Loss:  0.27775296568870544
Epoch:  8 Loss:  0.3677080273628235
Epoch:  9 Loss:  0.20003058016300201
Epoch:  10 Loss:  0.39250683784484863
Epoch:  11 Loss:  0.18470661342144012
Epoch:  12 Loss:  0.2395590990781784
Epoch:  13 Loss:  0.3973442316055298
Epoch:  14 Loss:  0.2830410599708557


/var/folders/s0/xcg0d2_s6tg_2v3r53qlxyl40000gn/T/ipykernel_1681/2898410133.py:15: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(y_pred, y_test)
/var/folders/s0/xcg0d2_s6tg_2v3r53qlxyl40000gn/T/ipykernel_1681/2898410133.py:15: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(y_pred, y_test)


Epoch:  15 Loss:  0.4305039048194885
Epoch:  16 Loss:  0.23616307973861694
Epoch:  17 Loss:  0.2730450928211212
Epoch:  18 Loss:  0.2649914622306824
Epoch:  19 Loss:  0.1844264566898346


In [127]:
predictions = []
for n in X_Test:
    predictions.append(model(n).item())

print(predictions)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

[0.14762558043003082, 0.07900910079479218, 0.04004330933094025, 0.20092009007930756, 0.22963528335094452, 0.31854379177093506, 0.18635831773281097, 0.18552222847938538, 0.29150834679603577, 0.20718184113502502, 0.1761801838874817, 0.0839732214808464, 0.21467538177967072, 0.03354013338685036, 0.07378937304019928, 0.21012212336063385, 0.14148302376270294, 0.25140470266342163, 0.18833351135253906, 0.09770681709051132, 0.0478653758764267, 0.37452176213264465, 0.17743712663650513, 0.24728186428546906, 0.07071051001548767, 0.06478803604841232, 0.2478446066379547, 0.23849919438362122, 0.10596396028995514, 0.1398271918296814, 0.06339913606643677, 0.1771005541086197, 0.1499013602733612, 0.1670031100511551, 0.1532597690820694, 0.27352985739707947, 0.18435579538345337, 0.26204344630241394, 0.21711009740829468, 0.1761801838874817, 0.1221110075712204, 0.17030152678489685, 0.11085707694292068, 0.18375957012176514, 0.08114569634199142, 0.21711009740829468, 0.0880182608962059, 0.1761801838874817, 0.04